Read the state election data from the csv file and the electoral college data from the JSON file

In [ ]:
import numpy as np
import pandas as pd
import csv

election_data = pd.read_csv('election-rows.csv', header = [0])
election_data

Now read the JSON file with the electoral college data.  The JSON file is something of a variant: its structure is as a dictionary of list of lists, where the first list gives the column order for that year and the remainder the values, e.g., {"1828": \[\['Party', 'Votes'\], \['Democratic', 200], \['Whig': 100\]\]}.  This is as opposed to the more reasonable {"1828": {"Whig": 100}, {"Democratic": 200}}.  As it happens, I know in this case it's Party first, so I will just hard-code that in this case.

In [ ]:
import json
f = open('electoral-college.json', 'r')
buffer = f.read()
f.close()
data = json.loads(buffer)
result = []
for year in data:
    year_record = data[year]
    year = int(year)
    for record in year_record[1:]:
        result.append([year, record[0], record[1]])
electoral_college_data = pd.DataFrame(result, columns=['Year', 'Party', 'Votes'])
electoral_college_data

key_to_columns is one of the major routines that we will need to prepare a flat table to be a chart.  What we have is a set of records with the fields (in this case) (state, year, candidate, party, votes, pct), and what we want (say) is a table of the form (year, republican votes, democrat votes).  republican, democrat are values in the party field.
Arguments:
* df: data frame in the flat form
* key_name: field to extract the values from
* merge_name: the field which is common to both (in the example above, year)
* rename_value_name: the field which is to be extracted and renamed for the value of the key name.  In the example above, votes is renamed republican or democrat

In [ ]:
def key_to_columns(df, key_name, merge_name, rename_value_name):
    key_values = set(df[key_name].tolist())
    frames = []
    for value in key_values:
        value_record_frame = df[df[key_name] == value][[merge_name, rename_value_name]].rename(columns = {rename_value_name: value})
        frames.append(value_record_frame)
    result = frames[0]
    for frame in frames[1:]:
        result = pd.merge(result, frame, on=merge_name, how='outer')
    return result


Get the data for the year.  This returns a table (data frame) with fields (Party, State, Pct), which shows the percentage each party got in the selected year.  This colors the map (not quite) in the demo

In [ ]:
def get_data_for_year(year):
    data = election_data[election_data['Year'] == year]
    return key_to_columns(data, 'Party', 'State', 'Pct')
get_data_for_year(1960)

Get the data for the state year.  This returns a table (data frame) with fields (Candidate, Party, Votes), which shows the votes each candidate  got in the selected year and state.  This fills the bottom-left image chart in the demo

In [ ]:
def get_data_for_year_and_state(year, state):
    return election_data[(election_data['Year'] == year)  & (election_data['State'] == state)][['Candidate', 'Party', 'Votes']]
get_data_for_year_and_state(1904, 'Illinois')

Get the historical data for each state.  This returns a data frame with fields (Year, Party, Pct Vote).  This is the data for the line chart in the bottom right.

In [ ]:
def get_data_for_state(state):
    return key_to_columns(election_data[election_data['State'] == state], 'Party', 'Year', 'Pct') 
get_data_for_state('Illinois')

Get the electoral college data for a year.  This is really simple, doesn't even need a function

In [ ]:
electoral_college_data[electoral_college_data['Year'] == 1828][['Party', 'Votes']]

And, finally, the data for the pie chart in the top right

In [ ]:
def get_summary_for_year(year):
    data = election_data[(election_data['Year'] == year) & (election_data['State'] == 'Nationwide') & (election_data['Party'] != 'Total')][['Party', 'Pct']]
    return data
get_summary_for_year(1960)